#  NLP - Classificador de chamados

---





---
Classificação de Chamados com NLP

A QuantumFinance possui um canal de atendimento via chat no qual os clientes descrevem, em texto livre, dúvidas e problemas relacionados aos seus serviços. Para tornar o atendimento mais eficiente, é necessário classificar automaticamente esses chamados por assunto e direcioná-los às áreas responsáveis.

Neste projeto, foi desenvolvido um classificador supervisionado de textos, aplicando técnicas de Processamento de Linguagem Natural (NLP/PLN), vetorização textual (n-gramas e embeddings) e modelos de Machine Learning. O modelo é treinado e avaliado utilizando um dataset rotulado, com divisão de 75% para treino e 25% para teste (random_state = 42).

Diferentes abordagens de pré-processamento, vetorização e modelagem foram testadas, e o desempenho foi avaliado com foco no F1 Score, métrica adequada para problemas de classificação com múltiplas classes. O modelo final atinge F1 Score superior a 75%, demonstrando viabilidade para uso em um cenário real de triagem automática de chamados.l de reprovação.

**[1] = ​https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv**

**[F1 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)** com average='weighted'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import requests

url = "https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv"
print(requests.get(url).text[:500])


id_reclamacao;data_abertura;categoria;descricao_reclamacao
3229299;2019-05-01T12:00:00-05:00;Hipotecas / EmprÃ©stimos;"Bom dia, meu nome Ã© xxxx xxxx e agradeÃ§o se vocÃª puder me ajudar a acabar com os serviÃ§os de membro do cartÃ£o bancÃ¡rio.
Em 2018, escrevi para Chase solicitar verificaÃ§Ã£o da dÃ­vida e o que eles me enviaram uma declaraÃ§Ã£o que nÃ£o Ã© aceitÃ¡vel. Estou pedindo ao banco que valide a dÃ­vida. Em vez disso, recebi e -mails todos os meses, tentando coletar uma dÃ­vida.
Ten


In [ ]:
# fazendo o resquest para ver as primeiras linhas do documento podemos ver que o separador utilizado é ';'

url = "https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv"
df = pd.read_csv(url, sep=';', encoding='utf-8')

# Verificar as primeiras linhas
df.head()

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo..."
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti..."
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos..."


O objetivo desse projeto é criar um classificador que assuntos. Para isso serão utilizadas técnicas supervisionadas, que necessitam de dados previamente rotulados, de maneira que as únicas colunas que serão utilizadas nesse processo são as colunas categoria (ou rótulo) e descrição.

In [ ]:
df = df[['descricao_reclamacao','categoria']]
df.head()

,descricao_reclamacao,categoria
0,"Bom dia, meu nome é xxxx xxxx e agradeço se vo...",Hipotecas / Empréstimos
1,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...,Cartão de crédito / Cartão pré-pago
2,O cartão Chase foi relatado em xx/xx/2019. No ...,Cartão de crédito / Cartão pré-pago
3,"Em xx/xx/2018, enquanto tentava reservar um ti...",Cartão de crédito / Cartão pré-pago
4,"Meu neto me dê cheque por {$ 1600,00} Eu depos...",Serviços de conta bancária


###**Area de desenvolvimento e validações**
Uma maneira de vetorizar texto é utilizando o conceito de bag of words, onde é feito um mapeamento com todas as pelavras encontradas no documento, que represetarão índices de posição de um vetor. Com isso cada expressão é mapeada e as palavras contidas nela são contadas, sendo que a frequência de cada palavra será atribuída ao indíce equivalente no vetor. É um conceito simples, porém limitado, já que ignora ordem e relações semânticas.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(1,1))
vect.fit(df.descricao_reclamacao)
count_vect = vect.transform(df.descricao_reclamacao)

pd.DataFrame(count_vect.toarray(), columns=vect.get_feature_names_out())

,00,000,0000,0000000,001,003,003933,004,0073,01,...,úmida,úmidas,úmido,úmidos,única,únicas,único,únicos,úteis,útil
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21067,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21068,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
21069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21070,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Uma característica desse procedimento é que resultado em uma matriz esparsa, onde a maioria das posições são preenchidas por zero. Uma possibilidade seria utilizar bi ou trigramas (agrupamentos de 2 ou 3 palavras) para ampliar esse estudo.

In [ ]:
count_vect

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 2491849 stored elements and shape (21072, 42401)>

Existem muitos termos que contém números, que não são significativos para a classificação das reclamações, então iremos refazer esse procedimento porém incluindo uma etapa inicial para remoção de todos os termos numéricos. Além disso iremos também fazer a remoção de stop words, caracteres não alfabéticos e de termos que contenham xx e XX, pois não há palavras na língua brasileira que com 'xx' e aqui x foi usado para substituir campos pessoais (como nomes e número de telefones).

In [ ]:
df['descricao_limpo'] = df['descricao_reclamacao'].str.replace(
    r'\b\w*(?:\d|x{2}|X{2}|/{1,}|$)\w*\b',
    '',
    regex=True
)

# Limpar espaços extras
df['descricao_limpo'] = df['descricao_limpo'].str.replace(r'\s+', ' ', regex=True).str.strip()

print(df)

                                    descricao_reclamacao  \
0      Bom dia, meu nome é xxxx xxxx e agradeço se vo...   
1      Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...   
2      O cartão Chase foi relatado em xx/xx/2019. No ...   
3      Em xx/xx/2018, enquanto tentava reservar um ti...   
4      Meu neto me dê cheque por {$ 1600,00} Eu depos...   
...                                                  ...   
21067  Depois de ser um cliente de cartão de persegui...   
21068  Na quarta -feira, xx/xx/xxxx, liguei para o Ch...   
21069  Não estou familiarizado com o XXXX Pay e não e...   
21070  Eu tive crédito impecável por 30 anos. Eu tive...   
21071  Mais de 10 anos atrás, encerrei minhas contas ...   

                                 categoria  \
0                  Hipotecas / Empréstimos   
1      Cartão de crédito / Cartão pré-pago   
2      Cartão de crédito / Cartão pré-pago   
3      Cartão de crédito / Cartão pré-pago   
4               Serviços de conta bancária   
...

Para limpar ainda mais o texto vamos remover todas as pontuações e demais símbolos que encontrarmos, tais com cifrões e sinais de porcentagem. Para isso vamos transformar todas as letras em minúsculas e criar uma lista com os símbolos e pontuação que queremos excluir.

In [ ]:
df['descricao_limpo'] = df['descricao_limpo'].str.lower()
print(df['descricao_limpo'].head())


0    bom dia, meu nome é e agradeço se você puder m...
1    atualizei meu cartão em e fui informado pelo a...
2    o cartão chase foi relatado em . no entanto, o...
3    em , enquanto tentava reservar um ticket , me ...
4    meu neto me dê cheque por {$ ,} eu depositei -...
Name: descricao_limpo, dtype: object


In [ ]:
# Concatenar todos os textos em uma única string
texto_unico = ''.join(df['descricao_limpo'].astype(str))

# Pegar conjunto de caracteres únicos e ordenar
chars_unicos = sorted(set(texto_unico))

print(chars_unicos)


[' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '°', 'º', 'à', 'á', 'â', 'ã', 'ç', 'é', 'ê', 'í', 'ó', 'ô', 'õ', 'ú', 'ü', '\u200b', '–', '“', '”', '•', '…', '−']


In [ ]:
import unicodedata

def is_letra(char):
    # Pega a categoria Unicode do caractere, tipo 'Ll' (letra minúscula), 'Lu' (maiúscula), etc.
    return unicodedata.category(char).startswith('L')

# Filtrar só os caracteres que NÃO são letras
chars_nao_letras = [c for c in chars_unicos if not is_letra(c)]

print(chars_nao_letras)


[' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '°', '\u200b', '–', '“', '”', '•', '…', '−']


In [ ]:
import re
# Escapar caracteres especiais para regex (ex: $, ., ?, etc.)
chars_escapados = [re.escape(c) for c in chars_nao_letras]

# Criar a classe regex: exemplo -> r'[$%\,!\?\.\/]'
classe_regex = f"[{''.join(chars_escapados)}]"
classe_regex

'[\\ !"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~°\u200b–“”•…−]'

Ainda existem muitos caracteres não alfabéticos, por isso criamos uma classe regex com todos esses caracteres e agora vamos removê-los.

In [ ]:
df['descricao_limpo'] = df['descricao_limpo'].str.replace(classe_regex, ' ', regex=True)
df['descricao_limpo'] = df['descricao_limpo'].str.replace(r'\s+', ' ', regex=True).str.strip()

df.head()

,descricao_reclamacao,categoria,descricao_limpo
0,"Bom dia, meu nome é xxxx xxxx e agradeço se vo...",Hipotecas / Empréstimos,bom dia meu nome é e agradeço se você puder me...
1,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...,Cartão de crédito / Cartão pré-pago,atualizei meu cartão em e fui informado pelo a...
2,O cartão Chase foi relatado em xx/xx/2019. No ...,Cartão de crédito / Cartão pré-pago,o cartão chase foi relatado em no entanto o pe...
3,"Em xx/xx/2018, enquanto tentava reservar um ti...",Cartão de crédito / Cartão pré-pago,em enquanto tentava reservar um ticket me depa...
4,"Meu neto me dê cheque por {$ 1600,00} Eu depos...",Serviços de conta bancária,meu neto me dê cheque por eu depositei o na mi...


Agora que as descrições já estão limpas de pontuações, caracteres especiais e dígitos, vamos remover stopwords.

In [ ]:
import nltk, string
nltk.download('stopwords'); nltk.download('punkt_tab')
stopwords = nltk.corpus.stopwords.words('portuguese')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# customização: manter 'muito' e 'não'; e adicionar termos de domínio
stopwords_new = [x for x in stopwords if x not in ('muito','não')]

In [ ]:
from nltk.tokenize import word_tokenize

# Garante que o tokenizador Punkt está disponível
import nltk
nltk.download('punkt', quiet=True)

# Função para remover stopwords
def remove_stopwords(text):
    tokens = word_tokenize(text.lower())             # tokeniza e coloca em minúsculas
    tokens = [t for t in tokens if t not in stopwords_new]  # remove stopwords
    return " ".join(tokens)                          # junta de volta em string

# Aplica a função à coluna desejada
df['descricao_sem_stopwords'] = df['descricao_limpo'].apply(remove_stopwords)


In [ ]:
df.head()

,descricao_reclamacao,categoria,descricao_limpo,descricao_sem_stopwords
0,"Bom dia, meu nome é xxxx xxxx e agradeço se vo...",Hipotecas / Empréstimos,bom dia meu nome é e agradeço se você puder me...,bom dia nome agradeço puder ajudar acabar serv...
1,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...,Cartão de crédito / Cartão pré-pago,atualizei meu cartão em e fui informado pelo a...,atualizei cartão informado agente fez atualiza...
2,O cartão Chase foi relatado em xx/xx/2019. No ...,Cartão de crédito / Cartão pré-pago,o cartão chase foi relatado em no entanto o pe...,cartão chase relatado entanto pedido fraudulen...
3,"Em xx/xx/2018, enquanto tentava reservar um ti...",Cartão de crédito / Cartão pré-pago,em enquanto tentava reservar um ticket me depa...,enquanto tentava reservar ticket deparei ofert...
4,"Meu neto me dê cheque por {$ 1600,00} Eu depos...",Serviços de conta bancária,meu neto me dê cheque por eu depositei o na mi...,neto dê cheque depositei conta chase fundo lim...


O próximo passa será dividir os dados em conjuntos de treino e teste. A limpeza inicial dos dados deve ser feita em todo o dataset, porém o ajuste da vetorização deve ser feito com os dados do conjunto de treino.

In [ ]:
df_final = df[['descricao_sem_stopwords', 'categoria']].rename(
    columns={'descricao_sem_stopwords': 'descricao'}
)
df_final.head()

,descricao,categoria
0,bom dia nome agradeço puder ajudar acabar serv...,Hipotecas / Empréstimos
1,atualizei cartão informado agente fez atualiza...,Cartão de crédito / Cartão pré-pago
2,cartão chase relatado entanto pedido fraudulen...,Cartão de crédito / Cartão pré-pago
3,enquanto tentava reservar ticket deparei ofert...,Cartão de crédito / Cartão pré-pago
4,neto dê cheque depositei conta chase fundo lim...,Serviços de conta bancária


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df_final['descricao'],
    df_final['categoria'],
    test_size=0.25,
    random_state=42
)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Cria o vetorizar para unigramas
vectorizer = CountVectorizer(ngram_range=(1,1))

# Ajusta (fit) apenas no conjunto de treino
x_train_bow = vectorizer.fit_transform(x_train)

# Transforma o conjunto de teste usando o mesmo vocabulário
x_test_bow = vectorizer.transform(x_test)


Agora vamos utilizar regressão logística para criar um classificador.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Cria e treina o modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train_bow, y_train)

# Faz previsões
y_pred = model.predict(x_test_bow)

# Avalia o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 0.8992027334851936

Relatório de Classificação:
                                      precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.90      0.91      0.91      1290
            Hipotecas / Empréstimos       0.91      0.93      0.92       922
                             Outros       0.88      0.87      0.88       549
       Roubo / Relatório de disputa       0.88      0.87      0.87      1204
         Serviços de conta bancária       0.91      0.91      0.91      1303

                           accuracy                           0.90      5268
                          macro avg       0.90      0.90      0.90      5268
                       weighted avg       0.90      0.90      0.90      5268



Obtivemos um bom valor de acurácia, e valores equilibrados de precision e recall, o que mostra que o classificador tem uma boa consistência geral.

Para ver se é possível aumentar a acurácia, vamos fazer a lematização da nossa base de dados, e acrescentar o uso de bigramas.

In [ ]:
import spacy

# Carrega o modelo de linguagem em português
nlp = spacy.load("pt_core_news_sm")

# Função para lematizar um texto
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

# Aplica a lematização à coluna 'descricao'
df_final['descricao'] = df_final['descricao'].apply(lemmatize_text)


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df_final['descricao'],
    df_final['categoria'],
    test_size=0.25,
    random_state=42
)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Cria o vetorizar para unigramas
vectorizer = CountVectorizer(ngram_range=(1,2))

# Ajusta (fit) apenas no conjunto de treino
x_train_bow = vectorizer.fit_transform(x_train)

# Transforma o conjunto de teste usando o mesmo vocabulário
x_test_bow = vectorizer.transform(x_test)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Cria e treina o modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train_bow, y_train)

# Faz previsões
y_pred = model.predict(x_test_bow)

# Avalia o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 0.9143887623386484

Relatório de Classificação:
                                      precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.92      0.92      0.92      1290
            Hipotecas / Empréstimos       0.91      0.95      0.93       922
                             Outros       0.90      0.90      0.90       549
       Roubo / Relatório de disputa       0.90      0.87      0.89      1204
         Serviços de conta bancária       0.93      0.93      0.93      1303

                           accuracy                           0.91      5268
                          macro avg       0.91      0.91      0.91      5268
                       weighted avg       0.91      0.91      0.91      5268



Ao utilizar tanto unigramas quanto bigramas conseguimos fazer com que o modelo melhorasse a sua acurácia.

Apesar de ter tido bons resultados, o conceito de BoW é bastante limitado já que apenas conta as palavras, sem interpretar quais palavras tem mais relevância no documento. Por isso vamos testar também a vetorização dos textos utilizando TF-IDF.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Cria o vetorizar para unigramas + bigramas
vectorizer = TfidfVectorizer(ngram_range=(1,2))

# Ajusta apenas no treino
x_train_tfidf = vectorizer.fit_transform(x_train)

# Transforma o teste com o mesmo vocabulário
x_test_tfidf = vectorizer.transform(x_test)


In [ ]:
# Cria e treina o modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train_tfidf, y_train)

# Faz previsões
y_pred = model.predict(x_test_tfidf)

# Avalia o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

Acurácia: 0.904707668944571

Relatório de Classificação:
                                      precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.91      0.91      0.91      1290
            Hipotecas / Empréstimos       0.92      0.92      0.92       922
                             Outros       0.93      0.83      0.87       549
       Roubo / Relatório de disputa       0.88      0.88      0.88      1204
         Serviços de conta bancária       0.91      0.94      0.92      1303

                           accuracy                           0.90      5268
                          macro avg       0.91      0.90      0.90      5268
                       weighted avg       0.91      0.90      0.90      5268



A acurácia foi menor em relação a BoW com unigramas e bigramas. Uma possibilidade ao usar TF-IDF é que palavras mais comuns acabam tendo pesos menores, mas podem ser que tenham alta relevância dentro do contexto do documento em análise. Também é possível que a utilização de unigramas e bigramas tenha aumentado muito a dimensionalidade da matriz e levado a um leve overfitting no treino, por isso vamos refazer essa análise usando apenas unigramas:

In [ ]:
# Cria o vetorizar para unigramas
vectorizer = TfidfVectorizer(ngram_range=(1,1))

# Ajusta apenas no treino
x_train_tfidf = vectorizer.fit_transform(x_train)

# Transforma o teste com o mesmo vocabulário
x_test_tfidf = vectorizer.transform(x_test)

In [ ]:
# Cria e treina o modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train_tfidf, y_train)

# Faz previsões
y_pred = model.predict(x_test_tfidf)

# Avalia o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

Acurácia: 0.9117312072892938

Relatório de Classificação:
                                      precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.93      0.91      0.92      1290
            Hipotecas / Empréstimos       0.91      0.93      0.92       922
                             Outros       0.91      0.87      0.89       549
       Roubo / Relatório de disputa       0.89      0.89      0.89      1204
         Serviços de conta bancária       0.92      0.93      0.93      1303

                           accuracy                           0.91      5268
                          macro avg       0.91      0.91      0.91      5268
                       weighted avg       0.91      0.91      0.91      5268



Aqui a acurácia teve uma melhora, o que é bastante interessante considerando que ao usar BoW a acurácia melhorou com a adição de bigramas, porém aqui foi maior com o uso exclusivo de unigramas.

Agora vamos trabalhar com word embeddings, inicialmente utilizaremos embeddings já treinadas para o português.

In [ ]:
# Instalação do pacote Gensim e dependência
!pip install gensim --quiet

In [ ]:
import gensim
print(gensim.__version__)

4.4.0


In [ ]:
# dependências de bibliotecas que vamos utilizar na Demo
!python -m spacy download pt_core_news_sm --quiet
!python -m spacy download pt_core_news_lg --quiet
!python -m spacy download en_core_web_lg --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 44.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 1.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 3.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load(

In [ ]:
# Download do arquivo no repositório do professor
!wget 'https://dados-ml-pln.s3-sa-east-1.amazonaws.com/cbow_s300.zip'
!ls -la
# veja o nome do arquivo compactado salvo pelo download

--2025-10-26 18:36:59--  https://dados-ml-pln.s3-sa-east-1.amazonaws.com/cbow_s300.zip
Resolving dados-ml-pln.s3-sa-east-1.amazonaws.com (dados-ml-pln.s3-sa-east-1.amazonaws.com)... 16.12.0.6, 16.12.0.30, 3.5.232.223, ...
Connecting to dados-ml-pln.s3-sa-east-1.amazonaws.com (dados-ml-pln.s3-sa-east-1.amazonaws.com)|16.12.0.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 929305948 (886M) [application/zip]
Saving to: ‘cbow_s300.zip’

cbow_s300.zip       100%[===================>] 886.25M  18.7MB/s    in 49s     

2025-10-26 18:37:48 (18.1 MB/s) - ‘cbow_s300.zip’ saved [929305948/929305948]

total 907548
drwxr-xr-x 1 root root      4096 Oct 26 18:36 .
drwxr-xr-x 1 root root      4096 Oct 26 15:09 ..
-rw-r--r-- 1 root root 929305948 Feb 16  2021 cbow_s300.zip
drwxr-xr-x 4 root root      4096 Oct 23 13:40 .config
drwxr-xr-x 1 root root      4096 Oct 23 13:40 sample_data


In [ ]:
# Descompactação do arquivo
!unzip 'cbow_s300.zip' # subistitua com nome do arquivo
!ls -la

Archive:  cbow_s300.zip
  inflating: cbow_s300.txt           
total 3501348
drwxr-xr-x 1 root root       4096 Oct 26 18:37 .
drwxr-xr-x 1 root root       4096 Oct 26 15:09 ..
-rw-r--r-- 1 root root 2656045531 Oct  4  2018 cbow_s300.txt
-rw-r--r-- 1 root root  929305948 Feb 16  2021 cbow_s300.zip
drwxr-xr-x 4 root root       4096 Oct 23 13:40 .config
drwxr-xr-x 1 root root       4096 Oct 23 13:40 sample_data


In [ ]:
# Load do modelo pelo Gensim
from gensim.models import KeyedVectors

model_cbow = KeyedVectors.load_word2vec_format('cbow_s300.txt')

In [ ]:
# model_cbow.save("cbow_s300.kv")
## e no futuro:
# model_cbow = KeyedVectors.load("cbow_s300.kv", mmap='r')


In [ ]:
model_cbow

Agora esse modelo será utilizado para converter as palavras em vetores, e cada linha (que representa uma descrição) será convertida em um vetor por meio da média dos vetores.

In [ ]:
# 1) texto -> tokens (split simples é suficiente se já está limpo)
tok_train = x_train.astype(str).apply(str.split)
tok_test  = x_test.astype(str).apply(str.split)

# 2) função: média dos vetores de palavras conhecidas
def doc_mean_vector(tokens, kv):
    vecs = [kv[t] for t in tokens if t in kv]     # ignora OOV
    if not vecs:
        return np.zeros(kv.vector_size, dtype=float)
    return np.mean(vecs, axis=0)

# 3) gera matrizes de embeddings
Xtr = np.vstack(tok_train.apply(lambda ts: doc_mean_vector(ts, model_cbow)))
Xte = np.vstack(tok_test.apply (lambda ts: doc_mean_vector(ts, model_cbow)))


In [ ]:
# Cria e treina o modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(Xtr, y_train)

# Faz previsões
y_pred = model.predict(Xte)

# Avalia o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

Acurácia: 0.8462414578587699

Relatório de Classificação:
                                      precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.88      0.85      0.87      1290
            Hipotecas / Empréstimos       0.83      0.87      0.85       922
                             Outros       0.84      0.76      0.80       549
       Roubo / Relatório de disputa       0.82      0.81      0.81      1204
         Serviços de conta bancária       0.85      0.89      0.87      1303

                           accuracy                           0.85      5268
                          macro avg       0.84      0.84      0.84      5268
                       weighted avg       0.85      0.85      0.85      5268



Utilizando um modelo de embeddings pré-treinado obtivemos uma acurácia de 0,85, que é inferior ao que foi obtido utilizando BoW. Por isso faremos mais um teste criando embeddings a partir do documento. Um modelo treinado de forma mais genérica pode deixar algumas palavras próximas, que durante um treino com vocabulário focado não ficariam tão próximas, justamente por estar tratando de uma quantidade tão vasta de observações.

Por fim vamos utilizar um modelo mais complexo que ao invés de gerar embeddings palavra a palavra e efetuar uma média para ter um embedding da sentença, já extrai diretamente embeddings de uma sentença, dando um vetor contextual da sentença.

In [ ]:
y_pred[3]

'Roubo / Relatório de disputa'

In [ ]:
!pip install sentence-transformers==3.2.1 transformers==4.46.3 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.7 MB/s eta 0:00:00


In [ ]:
import sentence_transformers
print(sentence_transformers.__version__)

/usr/local/lib/python3.12/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


3.2.1


In [ ]:
from sentence_transformers import SentenceTransformer

st = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')


In [ ]:
emb_train = st.encode(x_train.tolist(), batch_size=64, show_progress_bar=True, normalize_embeddings=True)
emb_test  = st.encode(x_test.tolist(),  batch_size=64, show_progress_bar=True, normalize_embeddings=True)

Batches:   0%|          | 0/247 [00:00<?, ?it/s]

Batches:   0%|          | 0/83 [00:00<?, ?it/s]

In [ ]:
# Cria e treina o modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(emb_train, y_train)

# Faz previsões
y_pred = model.predict(emb_test)

# Avalia o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

Acurácia: 0.8006833712984055

Relatório de Classificação:
                                      precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.82      0.81      0.82      1290
            Hipotecas / Empréstimos       0.82      0.86      0.84       922
                             Outros       0.77      0.69      0.73       549
       Roubo / Relatório de disputa       0.75      0.75      0.75      1204
         Serviços de conta bancária       0.83      0.85      0.84      1303

                           accuracy                           0.80      5268
                          macro avg       0.80      0.79      0.79      5268
                       weighted avg       0.80      0.80      0.80      5268



###**Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.

O modelo precisar atingir um score na métrica F1 Score superior a 75%.

**Atenção:**
- **Implemente aqui apenas os scripts que fazem parte do modelo campeão.**
- **Execute o pipeline do modelo campeão completamente para garantir que não tetá erros no script.**


**Pré-processamento**

Várias etapas iniciais foram efetuadas a fim de tornar os dados mais fáceis de serem manipulados e excluir informações que não são relevantes. Todos os caracteres numéricos, pontuações, caracteres especiais (não alfabéticos) foram removidos, além de stopwords e palavras que continham xx, já que estas estavam substituindo informações pessoais no documento. Essa etapa foi feita com todos os dados, antes da separação em conjuntos de treino e teste.

In [ ]:
df_final.head()

,descricao,categoria
0,bom dia nome agradeço puder ajudar acabar serv...,Hipotecas / Empréstimos
1,atualizei cartão informar agente fazer atualiz...,Cartão de crédito / Cartão pré-pago
2,cartão chase relatar entanto pedir fraudulento...,Cartão de crédito / Cartão pré-pago
3,enquanto tentar reservar ticket deparar oferta...,Cartão de crédito / Cartão pré-pago
4,neto dê cheque depositei contar chase fundo Li...,Serviços de conta bancária


A fim de focar apenas nas técnicas de PLN todos os modelos testados foram ajustados utilizando regressão logística. As seguintes técnicas foram testadas, com os seus correspondentes valores F1 Score:


*   Bag of Words com unigramas - F1 Score = 0.90
*   Bag of Words com combinação de unigramas e bigramas - F1 Score = 0.91
*   TF-IDF - F1 Score = 0.90
*   Word2Vec (com o modelo pré-treinado cbow) - F1 Score = 0.85
*   Sentence Transformer (embedding contextual global) - F1 Score = 0.80



Segue abaixo o pipeline utilizado para obter o modelo com a melhor performance:

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df_final['descricao'],
    df_final['categoria'],
    test_size=0.25,
    random_state=42
)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Cria o vetorizar para unigramas
vectorizer = CountVectorizer(ngram_range=(1,2))

# Ajusta (fit) apenas no conjunto de treino
x_train_bow = vectorizer.fit_transform(x_train)

# Transforma o conjunto de teste usando o mesmo vocabulário
x_test_bow = vectorizer.transform(x_test)


In [ ]:
# Cria e treina o modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train_bow, y_train)

# Faz previsões
y_pred = model.predict(x_test_bow)

# Avalia o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

Acurácia: 0.9143887623386484

Relatório de Classificação:
                                      precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.92      0.92      0.92      1290
            Hipotecas / Empréstimos       0.91      0.95      0.93       922
                             Outros       0.90      0.90      0.90       549
       Roubo / Relatório de disputa       0.90      0.87      0.89      1204
         Serviços de conta bancária       0.93      0.93      0.93      1303

                           accuracy                           0.91      5268
                          macro avg       0.91      0.91      0.91      5268
                       weighted avg       0.91      0.91      0.91      5268



### Conclusão

O entendimento de que um método mais moderno e complexo será capaz de resolver um problema de maneira mais eficiente é bastante comum, porém essa intuição nem sempre se concretiza. No caso de uma base de dados simples, com descrições como nesse caso, o método BoW apresentou o melhor resultado, apesar de sua simplicidade. Quando comparado com os métodos mais avançado como de embeddings de palavras e de sentenças que foram treinado para situações mais generalistas acabam não performando tão bem para contextos mais específicos, visto que criam uma espécie de suavização semântica, justamente por apresentar esse contexto mais generalista, especialmente o SentenceTransformer, que é bom para identificação de similaridades globais, porém fraco para discriminação local.

 Para situações que podem ser resolvidade com o destaque de palavras-chave BoW performa muito bem, especialmente quando uma combinação de unigramas  e bigramas é utilizada.